### Load preprocessed data

In [1]:
import numpy as np
fh = np.load('../data/dataset.npz')
# We have a bunch of feature columns and last column is the y-target
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']

n_user = int(fh['n_user'])
n_item = int(fh['n_item'])
n_occu = int(fh['n_occu'])

### Define the MF Model

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss


class MF(nn.Module):
    itr = 0
    
    def __init__(self, n_user, n_item, n_occu, k=18, c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_bias = c_bias
        self.c_vector = c_vector
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
        self.bias_user = nn.Embedding(n_user, 1)
        self.bias_item = nn.Embedding(n_item, 1)
        self.bias = nn.Parameter(torch.ones(1))
        
        # **NEW: occupation vectors
        self.occu = nn.Embedding(n_occu, k)

    
    def __call__(self, train_x):
        user_id = train_x[:, 0]
        item_id = train_x[:, 1]
        vector_user = self.user(user_id)
        vector_item = self.item(item_id)
        bias_user = self.bias_user(user_id).squeeze()
        bias_item = self.bias_item(item_id).squeeze()
        biases = (self.bias + bias_user + bias_item)
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        
        # **NEW: occupation-item interaction
        occu_id = train_x[:, 3]
        vector_occu = self.occu(occu_id)
        oi_interaction = torch.sum(vector_user * vector_occu, dim=1)
        
        prediction = ui_interaction + oi_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
        loss_mse = F.mse_loss(prediction.squeeze(), target.squeeze())
        prior_bias_user =  l2_regularize(self.bias_user.weight) * self.c_bias
        prior_bias_item = l2_regularize(self.bias_item.weight) * self.c_bias
        prior_user =  l2_regularize(self.user.weight) * self.c_vector
        prior_item = l2_regularize(self.item.weight) * self.c_vector
        prior_occu = l2_regularize(self.item.weight) * self.c_vector
        total = loss_mse + prior_user + prior_item + prior_bias_item + prior_bias_user +  prior_occu
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total

### Train model

In [3]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError

from loader import Loader
from datetime import datetime

#### Hyperparameters

In [4]:
lr = 1e-2
k = 10
c_bias = 1e-6
c_vector = 1e-6
log_dir = 'runs/simple_mf_04_side_' + str(datetime.now()).replace(' ', '_')
print(log_dir)

runs/simple_mf_04_side_2018-08-19_11:04:04.996877


In [5]:
writer = SummaryWriter(log_dir=log_dir)
model = MF(n_user, n_item, n_occu, k=k, c_bias=c_bias, c_vector=c_vector, writer=writer)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)
train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)


def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluat.run(test_loader)
    metrics = evaluat.state.metrics
    avg_accuracy = metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} "
          .format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


model

MF(
  (user): Embedding(6041, 10)
  (item): Embedding(3953, 10)
  (bias_user): Embedding(6041, 1)
  (bias_item): Embedding(3953, 1)
  (occu): Embedding(21, 10)
)

#### Run model

In [6]:
trainer.run(train_loader, max_epochs=250)

Epoch[1] Iteration[400/879] Loss: 3.19
Epoch[1] Iteration[800/879] Loss: 1.54
Epoch[1] Validation MSE: 1.32 
Epoch[2] Iteration[1200/879] Loss: 1.18
Epoch[2] Iteration[1600/879] Loss: 0.91
Epoch[2] Validation MSE: 0.95 
Epoch[3] Iteration[2000/879] Loss: 0.82
Epoch[3] Iteration[2400/879] Loss: 0.89
Epoch[3] Validation MSE: 0.90 
Epoch[4] Iteration[2800/879] Loss: 0.93
Epoch[4] Iteration[3200/879] Loss: 0.91
Epoch[4] Validation MSE: 0.88 
Epoch[5] Iteration[3600/879] Loss: 0.84
Epoch[5] Iteration[4000/879] Loss: 0.76
Epoch[5] Validation MSE: 0.85 
Epoch[6] Iteration[4400/879] Loss: 0.77
Epoch[6] Iteration[4800/879] Loss: 0.73
Epoch[6] Iteration[5200/879] Loss: 0.78
Epoch[6] Validation MSE: 0.82 
Epoch[7] Iteration[5600/879] Loss: 0.74
Epoch[7] Iteration[6000/879] Loss: 0.73
Epoch[7] Validation MSE: 0.81 
Epoch[8] Iteration[6400/879] Loss: 0.71
Epoch[8] Iteration[6800/879] Loss: 0.72
Epoch[8] Validation MSE: 0.79 
Epoch[9] Iteration[7200/879] Loss: 0.74
Epoch[9] Iteration[7600/879] Loss:

Epoch[67] Validation MSE: 0.76 
Epoch[68] Iteration[59200/879] Loss: 0.68
Epoch[68] Iteration[59600/879] Loss: 0.77
Epoch[68] Validation MSE: 0.76 
Epoch[69] Iteration[60000/879] Loss: 0.63
Epoch[69] Iteration[60400/879] Loss: 0.71
Epoch[69] Validation MSE: 0.76 
Epoch[70] Iteration[60800/879] Loss: 0.66
Epoch[70] Iteration[61200/879] Loss: 0.72
Epoch[70] Validation MSE: 0.76 
Epoch[71] Iteration[61600/879] Loss: 0.66
Epoch[71] Iteration[62000/879] Loss: 0.70
Epoch[71] Iteration[62400/879] Loss: 0.71
Epoch[71] Validation MSE: 0.76 
Epoch[72] Iteration[62800/879] Loss: 0.65
Epoch[72] Iteration[63200/879] Loss: 0.72
Epoch[72] Validation MSE: 0.76 
Epoch[73] Iteration[63600/879] Loss: 0.72
Epoch[73] Iteration[64000/879] Loss: 0.71
Epoch[73] Validation MSE: 0.76 
Epoch[74] Iteration[64400/879] Loss: 0.67
Epoch[74] Iteration[64800/879] Loss: 0.64
Epoch[74] Validation MSE: 0.76 
Epoch[75] Iteration[65200/879] Loss: 0.63
Epoch[75] Iteration[65600/879] Loss: 0.70
Epoch[75] Validation MSE: 0.76

Epoch[132] Validation MSE: 0.76 
Epoch[133] Iteration[116400/879] Loss: 0.64
Epoch[133] Iteration[116800/879] Loss: 0.69
Epoch[133] Validation MSE: 0.76 
Epoch[134] Iteration[117200/879] Loss: 0.71
Epoch[134] Iteration[117600/879] Loss: 0.71
Epoch[134] Validation MSE: 0.76 
Epoch[135] Iteration[118000/879] Loss: 0.73
Epoch[135] Iteration[118400/879] Loss: 0.72
Epoch[135] Validation MSE: 0.76 
Epoch[136] Iteration[118800/879] Loss: 0.62
Epoch[136] Iteration[119200/879] Loss: 0.70
Epoch[136] Validation MSE: 0.76 
Epoch[137] Iteration[119600/879] Loss: 0.64
Epoch[137] Iteration[120000/879] Loss: 0.68
Epoch[137] Iteration[120400/879] Loss: 0.68
Epoch[137] Validation MSE: 0.76 
Epoch[138] Iteration[120800/879] Loss: 0.70
Epoch[138] Iteration[121200/879] Loss: 0.75
Epoch[138] Validation MSE: 0.76 
Epoch[139] Iteration[121600/879] Loss: 0.68
Epoch[139] Iteration[122000/879] Loss: 0.73
Epoch[139] Validation MSE: 0.76 
Epoch[140] Iteration[122400/879] Loss: 0.72
Epoch[140] Iteration[122800/879]

Epoch[196] Iteration[172000/879] Loss: 0.69
Epoch[196] Validation MSE: 0.76 
Epoch[197] Iteration[172400/879] Loss: 0.66
Epoch[197] Iteration[172800/879] Loss: 0.72
Epoch[197] Validation MSE: 0.76 
Epoch[198] Iteration[173200/879] Loss: 0.68
Epoch[198] Iteration[173600/879] Loss: 0.63
Epoch[198] Iteration[174000/879] Loss: 0.75
Epoch[198] Validation MSE: 0.76 
Epoch[199] Iteration[174400/879] Loss: 0.66
Epoch[199] Iteration[174800/879] Loss: 0.73
Epoch[199] Validation MSE: 0.76 
Epoch[200] Iteration[175200/879] Loss: 0.64
Epoch[200] Iteration[175600/879] Loss: 0.75
Epoch[200] Validation MSE: 0.76 
Epoch[201] Iteration[176000/879] Loss: 0.66
Epoch[201] Iteration[176400/879] Loss: 0.67
Epoch[201] Validation MSE: 0.76 
Epoch[202] Iteration[176800/879] Loss: 0.70
Epoch[202] Iteration[177200/879] Loss: 0.68
Epoch[202] Validation MSE: 0.76 
Epoch[203] Iteration[177600/879] Loss: 0.63
Epoch[203] Iteration[178000/879] Loss: 0.77
Epoch[203] Iteration[178400/879] Loss: 0.72
Epoch[203] Validation